## Print Top 10 Similar Frames
#### This is the output of cosine_similarity_search.py. The code access the embedding vectors stored in all csv files in the NPREmbeddings folder, calculate cosine similarity then get the indices of top N similar vectors.

In [1]:
import numpy as np
import pandas as pd
import csv
import os
from sklearn.metrics.pairwise import normalize, cosine_similarity

folder_path = 'NPREmbeddings'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]


# Load each CSV file and concatenate the DataFrames
df = pd.DataFrame()
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df_vid = pd.read_csv(file_path)

    # Add a 'Video_Name' column to identify each video
    video_name = os.path.splitext(csv_file)[0] 
    df_vid['Video_Name'] = video_name
    df = pd.concat([df, df_vid], ignore_index=True)

# Normalize embeddings
# Separate 'Frame_Name' column for later use
frame_names = df['Frame_Name']
df = df.drop(columns=['Frame_Name', 'Video_Name'])
df = pd.DataFrame(normalize(df, axis=1), columns=df.columns)

# Concatenate 'Frame_Name' column back to the DataFrame
df['Frame_Name'] = frame_names

def semantic_search(query_vector, dataframe, top_n=10):
    """
    Perform semantic search using cosine similarity.
    
    Parameters:
    - query_vector: The vector of the query.
    - dataframe: The DataFrame containing all vectors.
    - top_n: Number of similar vectors to retrieve.
    
    Returns:
    - top_n_indices: Indices of the top N similar vectors.
    """
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity([query_vector], dataframe.iloc[:, :-1].values)[0]

    # Get the indices of top N similar vectors
    top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]

    return top_n_indices

def generate_query_vector(csv_file, target_frame_name):
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)

        # Find the index of the frame_name in the header
        frame_name_index = header.index('Frame_Name')

        for row in reader:
            frame_name = row[0]
            if frame_name == target_frame_name:
                frame_data = list(map(float, row[1:]))  # Convert frame_data to float
                return frame_data

    return None

# Example usage:
# Assuming you have a query_vector (e.g., the vector of the user's query)
# and df is the DataFrame containing all video embeddings
# Specify the CSV file path and frame name
csv_file_path = 'NPREmbeddings/2000 Bodies Recovered After Dam Bursts In Derna Libya.csv'
frame_name = '2000_Bodies_Recovered_After_Dam_Bursts_In_Derna_Libya_frame_0000'
query_vector = generate_query_vector(csv_file_path, frame_name)

if query_vector is not None:
    top_similar_indices = semantic_search(query_vector, df)
    top_similar_videos = df.iloc[top_similar_indices]
    print(top_similar_videos)
else:
    print(f"Frame name '{frame_name}' not found in the CSV file.")


C:\Users\thaou\AppData\Local\Temp\ipykernel_776\3451894857.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_vid], ignore_index=True)


    Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0    0.045443   0.044564   0.043701   0.050924   0.039351   0.042087   
6    0.045107   0.045474   0.044699   0.049647   0.040392   0.042007   
7    0.044969   0.047844   0.042201   0.048919   0.038059   0.043054   
5    0.044425   0.045170   0.043319   0.051482   0.040085   0.042006   
13   0.044203   0.045178   0.043827   0.050559   0.040495   0.042899   
4    0.044543   0.044307   0.044359   0.050740   0.038902   0.042239   
1    0.045598   0.045650   0.045880   0.050546   0.040334   0.042043   
16   0.044065   0.045101   0.043593   0.052230   0.041126   0.043044   
12   0.044427   0.045124   0.044219   0.049521   0.040973   0.042277   
2    0.045179   0.045751   0.045950   0.050591   0.040414   0.040394   

    Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_503  Feature_504  \
0    0.044200   0.056195   0.044337   0.043860  ...     0.039878     0.043183   
6    0.042889   0.056260   0.045464   0.04321